In [1]:
!pip install pulp==3.2.0

In [ ]:
#import importlib.metadata

#version = importlib.metadata.version("rapids-dask-dependency")
#print(version)

25.6.0


In [2]:
!pip install --extra-index-url=https://pypi.nvidia.com cuopt-cu12==25.5.*

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [3]:
import pulp

In [4]:
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD', 'HiGHS', 'CUOPT']

In [5]:
import numpy as np
import pandas as pd
import time

BIG_M = 30000000
TN_DAILY_INTEREST_RATE = 0.00092

def model_problem(
    amount_of_days, amount_of_branches, amount_of_routes,
    route_branches_csv, cost_routes_csv, cash_in_branch_csv,
    box_amounts_csv, business_days_csv, collection_csv,
    last_days_collection=list(), extra_box_percent=0.0, daily_interest_rate=0.0,
    debug=False, solver='cbc', n_thr=4):
    """
    Función que modela y resuelve el problema de envío de camiones de acuerdo a los datos
    de entrada, que vienen en forma de CSVs.
    - amount_of_days: Cantidad de días de planeamiento.
    - amount_of_branches: Cantidad de sucursales en la planificación.
    - amount_of_routes: Cantidad de rutas diferentes comprendidas en la planificación.
    - *_csv: CSVs con los datos que deben incorporarse, ejemplos en carpeta 'data/'
    - last_days_collection: Lista de los días en los que se debe ir a buscar dinero. Suelen ser los últimos n
    - extra_box_percent: Porcentaje extra que se permite guardar de dinero en cada sucursal.
    - daily_interest_rate: Tasa diaria de interés, para incorporar costo financiero.
    - debug: Permite imprimir todas las variables del problema, default False.
    - scip: True si se utilizará SCIP para resolver, False si se usará CBC.
    """

    if daily_interest_rate < 0.0:
        print("Tasa de interes no puede ser menor a cero (al menos en Argentina...)")
        return None

    status = []
    variables = []
    msg_flag = False
    if debug:
        start = time.time()
        msg_flag = True

    routes_matrix = np.loadtxt(route_branches_csv, delimiter=",")
    separable = (routes_matrix.size == 1) or np.array_equal(routes_matrix, np.diag(np.diag(routes_matrix)))

    if separable:
        problems = np.arange(amount_of_branches)
    else:
        problems = [0]

    days = range(amount_of_days)
    branches = range(amount_of_branches)
    routes = range(amount_of_routes)

    #### DATOS

    # Sucursales en cada ruta
    route_branches_df = pd.read_csv(route_branches_csv, header=None)
    route_branches = {
        ridx: list(route_branches_df.iloc[ridx]) for ridx in routes
    }

    if debug:
        print("route_branches = {}".format(route_branches))

    # Costo de tomar cada ruta
    cost_routes_df = pd.read_csv(cost_routes_csv, header=None)
    cost_routes = {
        ridx: float(cost_routes_df.iloc[ridx,0]) for ridx in routes
    }

    if debug:
        print("cost_routes = {}".format(cost_routes))

    # Efectivo inicial en cada sucursal
    cash_branches_df = pd.read_csv(cash_in_branch_csv, header=None)
    first_cash_in_branch = [
        float(cash_branches_df.iloc[bidx,0]) for bidx in branches
    ]

    if debug:
        print("first_cash_in_branch = {}".format(first_cash_in_branch))

    # Efectivo máximo de cada buzón
    box_amounts_df = pd.read_csv(box_amounts_csv, header=None)
    box_max = [
        float(box_amounts_df.iloc[bidx,0]) for bidx in branches
    ]

    if debug:
        print("box_max = {}".format(box_max))

    # Días hábiles por ruta
    business_days_df = pd.read_csv(business_days_csv, header=None)
    business_days = {
        ridx: list(business_days_df.iloc[ridx]) for ridx in routes
    }

    if debug:
        print("business_days = {}".format(business_days))

    # Recaudacion por sucursal por dia
    collection_df = pd.read_csv(collection_csv, header=None, sep="\t")
    collection = {
        bidx: list(collection_df.iloc[bidx]) for bidx in branches
    }

    if debug:
        print("collection = {}".format(collection))

    Problems = []

    for prob in problems:
        problem = pulp.LpProblem("MinimizeCosts", pulp.LpMinimize)

        #### INCOGNITAS
        if separable:
            branches = [prob]
            routes = [prob]

        # Variables X: retira o no por día y por ruta.
        days_routes = {
            ridx: {
                didx: pulp.LpVariable(
                    "x_{}_{}".format(str(didx), str(ridx)),
                    cat=pulp.LpBinary,
                ) for didx in days
            } for ridx in routes
        }

        if debug:
            print("days_routes = {}".format(days_routes))

        # Variables E: efectivo por sucursal y por día
        # lowBound=0 --->  0 <= e[s,d]
        branch_cash = {
            bidx: {didx: pulp.LpVariable(
                    "e_{}_{}".format(str(bidx), str(didx)),
                    lowBound=0,
                    cat=pulp.LpContinuous,
                ) for didx in days
            } for bidx in branches
        }

        if debug:
            print("branch_cash = {}".format(branch_cash))

        # Variables T: efectivo retirado por sucursal por dia por ruta
        # lowBound=0 ---> 0 <= t[s,d,p]
        withdrawn_cash = {
            bidx: {
                didx: {
                    ridx: pulp.LpVariable(
                        "t_{}_{}_{}".format(bidx, didx, ridx),
                        lowBound=0,
                        cat=pulp.LpContinuous,
                    ) for ridx in routes
                 } for didx in days
            } for bidx in branches
        }

        if debug:
            print("withdrawn_cash = {}".format(withdrawn_cash))

        #### FUNCION OBJETIVO

        cost_function = None
        for index in days_routes.keys():
            cost_function += sum(days_routes[index].values()) * cost_routes[index]

        if daily_interest_rate > 0.0:
            cost_function += sum([first_cash_in_branch[bidx] * daily_interest_rate for bidx in branches])
            cost_function += sum([branch_cash[bidx][day] * daily_interest_rate for bidx in branches for day in days[:-1]])

        problem += cost_function

        #### RESTRICCIONES

        # e[s,1] == e0[s,1] + r[s,1] - sum <p> in P: (t[s,1,p])
        # e[s,d] == e[s,d-1] + r[s,d] - sum <p> in P: (t[s,d,p])
        # if amount_of_branches == 6:
        #     import ipdb; ipdb.set_trace()
        for bidx in branches:
            problem += branch_cash[bidx][0] == first_cash_in_branch[bidx] + collection[bidx][0] - sum([withdrawn_cash[bidx][0][route] for route in routes])
            for day in days[1:]:
                problem += (
                    branch_cash[bidx][day]
                    ==
                    branch_cash[bidx][day - 1] + collection[bidx][day] - sum([withdrawn_cash[bidx][day][route] for route in routes])
                )

        # forall <d,p> in D*P:
        #     sum <s> in S: (m[s,p]*t[s,d,p]) <= M * x[d,p];
        for day in days:
            for route in routes:
                problem += sum([route_branches[route][bidx] * withdrawn_cash[bidx][day][route] for bidx in branches]) <= BIG_M * days_routes[route][day]

        # forall <d,p> in D*P:
        #     sum <s> in S: ((1-m[s,p])*t[s,d,p]) == 0;
        for day in days:
            for route in routes:
                problem += sum([(1 - route_branches[route][bidx]) * withdrawn_cash[bidx][day][route] for bidx in branches]) == 0

        # forall <s,d> in S*D:
        #     e[s,d] <= b[s]*1.17;
        for day in days:
            for bidx in branches:
                problem += branch_cash[bidx][day] <= box_max[bidx] * (1.0 + extra_box_percent)

        # forall <s,d,p> in S*(D-{1})*P:
        #    t[s,d,p] <= e[s,d-1];
        for branch in branches:
            problem += sum([withdrawn_cash[branch][0][route] for route in routes]) <= first_cash_in_branch[branch]
            for day in days[1:]:
                problem += sum([withdrawn_cash[branch][day][route] for route in routes]) <= branch_cash[branch][day - 1]

        # forall <d,p> in D*P:
        #     x[d,p] <= h[d,p];
        for day in days:
            for route in routes:
                problem += days_routes[route][day] <= business_days[route][day]

        # recollection on D_q days
        if len(last_days_collection) > 0:
            for bidx in branches:
                problem += sum([
                    route_branches[route][bidx] * days_routes[route][day]
                    for route in routes for day in last_days_collection
                ]) >= 1

        solv = None

        if solver=='scip':
             #solv = pulp.apis.scip_api.SCIP_CMD(msg=msg_flag)
             solv = pulp.apis.SCIP_CMD(msg=msg_flag)
        elif solver == 'fscip':
            #solv = pulp.apis.fscip_api.FSCIP_CMD(msg=msg_flag)
            solv = pulp.apis.FSCIP_CMD(msg=msg_flag)
        elif solver == 'cbc':
            #solv = pulp.PULP_CBC_CMD(dual=1, strong=1, msg=msg_flag, presolve=1, threads=n_thr)
            solv = pulp.PULP_CBC_CMD(strong=1, msg=msg_flag, presolve=1, threads=n_thr)
        elif solver == "cuopt":
            solv = pulp.CUOPT(msg=msg_flag)
        else:
            print("WARNING: Unkown solver, defaulting to cbc")
            #solv = pulp.PULP_CBC_CMD(dual=1, strong=1, msg=msg_flag, presolve=1, threads=n_thr)
            solv = pulp.PULP_CBC_CMD(strong=1, msg=msg_flag, presolve=1, threads=n_thr)

        try:
            problem.solve(solver = solv)

        except Exception as e:
            print("Can't solve problem: {}".format(e))

        if debug:
            print("Solver took {} seconds.".format(time.time() - start))

        variables += problem.variables()

        cur_status = 'Error'

        for bidx in branches:
            if max(collection[bidx]) > box_max[bidx] * (1.0 + extra_box_percent):
                # error de buzon
                cur_status += ', capacidad de buzón superada'
                break

        if len(routes)==1:
            last_days_business = [d for d in last_days_collection if business_days[routes[0]][d]==1]
            if len(last_days_collection)>0 and len(last_days_business)==0:
                cur_status += ', día/s obligatorio/s infactible/s'

        if problem.status == 1:
            cur_status = 'Resuelto'

        status.append(cur_status)
        Problems.append(problem)

    return status, variables, Problems


In [17]:
import sys
import os
import itertools
from scipy.special import comb
import numpy as np
import pandas as pd
import time

#sys.path.append('../solverpulp')
#import model

def calculo_recaudaciones(prop_suc, collections, e_zero, buzones):
	# ToDo: propagar prop_suc
	return collections, e_zero, buzones

def calculo_ganancia(business_days, rutas, costos_rutas, interes, prop_suc, collections, e_zero, buzones, n_thr=4, solver='cbc', debug=False):
	data_dir = './data/'

	# Dias habiles por ruta
	habiles_csv_path = os.path.join(data_dir, "habiles.csv")
	business_days.to_csv(habiles_csv_path, header=False, index=False)

	# Datos de rutas
	rutas_csv_path = os.path.join(data_dir, "rutas.csv")
	rutas.to_csv(rutas_csv_path, header=False, index=False)

	# Costos por ruta
	costo_rutas_csv_path = os.path.join(data_dir, "costo_rutas.csv")
	costos_rutas.to_csv(costo_rutas_csv_path, header=False, index=False)

	collections, e_zero, buzones = calculo_recaudaciones(prop_suc, collections, e_zero, buzones)

	# Recaudaciones por sucursal
	recaudacion_csv_path = os.path.join(data_dir, "recaudacion.csv")
	collections.to_csv(recaudacion_csv_path, header=False, index=False, sep="\t")

	# Recaudacion inicial
	e_zero_csv_path = os.path.join(data_dir, "e0.csv")
	e_zero.to_csv(e_zero_csv_path, header=False, index=False)

	# Datos de buzones
	buzones_csv_path = os.path.join(data_dir, "buzon.csv")
	buzones.to_csv(buzones_csv_path, header=False, index=False)

	_,n_d = business_days.shape
	n_p,n_s = rutas.shape

	# Calcular con costo financiero
	status, variables, Problems = model_problem(
		n_d, n_s, n_p,
		rutas_csv_path, costo_rutas_csv_path, e_zero_csv_path,
		buzones_csv_path, habiles_csv_path, recaudacion_csv_path,
		daily_interest_rate=interes,n_thr=n_thr,solver=solver,debug=debug
	)
	try:
		costos_total_caso_financiero = sum([prob.objective.value() for prob in Problems])
	except:
		return -1

	# Calcular sin costo financiero
	status, variables, Problems = model_problem(
		n_d, n_s, n_p,
		rutas_csv_path, costo_rutas_csv_path, e_zero_csv_path,
		buzones_csv_path, habiles_csv_path, recaudacion_csv_path,
		daily_interest_rate=0.0,n_thr=n_thr,solver=solver,debug=debug
	)
	try:
		costos_logístico_sin_financiero = sum([prob.objective.value() for prob in Problems])
	except:
		return -1

	# Calcular costo financiero del caso logístico
	costos_financiero_logístico = e_zero.values.sum()
	for var in variables:
		var_id = var.name.split('_')[0]
		if var_id == 'e':
			suc,dia = var.name.split('_')[1:]
			if dia != n_d-1:
				costos_financiero_logístico += var.varValue
	costos_financiero_logístico *= interes

	costos_total_caso_logistico = costos_financiero_logístico + costos_logístico_sin_financiero

	ganancia = (costos_total_caso_logistico - costos_total_caso_financiero) / costos_total_caso_logistico

	return ganancia

def generar_escenarios(n_s,n_p,n):
	cant_rutas_posibles = 2**n_s - n_s - 1
	cant_rutas_elegir = n_p - n_s
	# Generar la lista de números que determinan cada ruta no trivial
	rutas_posibles = list(range(2**n_s))
	rutas_posibles.remove(0)
	for i in range(n_s):
		rutas_posibles.remove(2**i)
	# Generar listas de tamaño n_p-n_s de índices de esa lista
	escenarios_posibles = itertools.combinations(range(cant_rutas_posibles),cant_rutas_elegir)
	# Elegir n de esas listas
	N = int(comb(cant_rutas_posibles,cant_rutas_elegir))
	indices_escenarios = np.random.choice(N,n,replace=False)
	r = iter(range(N))
	# filter
	indices_rutas = [next(escenarios_posibles) for _ in range(N) if next(r) in indices_escenarios]
	# Generar las rutas
	escenarios = []
	fmt_str = "{:0"+str(n_s)+"b}"
	for indices in indices_rutas:
		rutas = [rutas_posibles[i] for i in indices]
		rutas = [fmt_str.format(ruta) for ruta in rutas]
		rutas = [list(map(int,ruta)) for ruta in rutas]
		rutas = np.vstack([np.eye(n_s),rutas])
		rutas = pd.DataFrame(rutas)
		escenarios.append(rutas)
	return escenarios

def agregar_resultado(exp_dict, collections_profiles, std_profiles, n_thr, solver, debug=False):
	data_dir = './data/'
	rutas_csv_path = os.path.join(data_dir, "rutas.csv")
	costo_rutas_csv_path = os.path.join(data_dir, "costo_rutas.csv")
	habiles_csv_path = os.path.join(data_dir, "habiles.csv")
	# generar semilla
	rand_seed = len(exp_dict)
	exp_dict[str(rand_seed)] = {}
	# generar perfiles aleatorios
	rng = np.random.default_rng(seed=rand_seed)
	profiles_names = ["constant", "V"]
	for collections,std,name in zip(collections_profiles,std_profiles, profiles_names):
		n_s,n_d = collections.shape
		#n_p,_ = rutas.shape
		n_p = 8
		e_zero = collections[:,0]
		rand_collect = rng.normal(loc=0.0,scale=std,size=collections.shape)
		rand_e_zero = rng.normal(loc=0,scale=std,size=e_zero.shape)
		# la std constante es 52% de la recaudación diaria
		# 1.9 asegura no tener recaudaciones negativas
		rand_collect = np.clip(rand_collect,-1.9*std,1.9*std)
		rand_e_zero = np.clip(rand_e_zero,-1.9*std,1.9*std)
		collections = collections + rand_collect
		e_zero = e_zero + rand_e_zero
		e_zero = pd.DataFrame(e_zero)
		collections = pd.DataFrame(collections)
		# guardar perfiles aleatorios
		# Recaudaciones por sucursal
		recaudacion_csv_path = os.path.join(data_dir, "recaudacion.csv")
		collections.to_csv(recaudacion_csv_path, header=False, index=False, sep="\t")
		# Recaudacion inicial
		e_zero_csv_path = os.path.join(data_dir, "e0.csv")
		e_zero.to_csv(e_zero_csv_path, header=False, index=False)
		exp_dict[str(rand_seed)][name] = {}
		for interes_anual in np.linspace(0,10,11):
			interes = (1+interes_anual/100)**(1/365)-1
			exp_dict[str(rand_seed)][name][str(interes_anual)] = {}
			#cantidad de recolecciones mensuales
			for b in range(4):
				buzones = np.ones(n_s) / (b+1)
				buzones = pd.DataFrame(buzones)
				# Datos de buzones
				buzones_csv_path = os.path.join(data_dir, "buzon.csv")
				buzones.to_csv(buzones_csv_path, header=False, index=False)
				print(f"Resolviendo caso {rand_seed} {name} {interes_anual} {b} ", end="")
				start = time.time()
				# Resolver problema
				status, variables, Problems = model_problem(
					n_d, n_s, n_p,
					rutas_csv_path, costo_rutas_csv_path, e_zero_csv_path,
					buzones_csv_path, habiles_csv_path, recaudacion_csv_path,
					daily_interest_rate=interes,n_thr=n_thr,solver=solver,debug=debug
				)
				tiempo = time.time()-start
				print(f"t={tiempo:.2f}")
				# import pdb;
				# pdb.set_trace()
				try:
					# Calcular costo total
					costo_total = sum([prob.objective.value() for prob in Problems])
					# Calcular costo financiero sin interés
					costo_financiero = e_zero.values.sum()
					for var in variables:
						var_id = var.name.split('_')[0]
						if var_id == 'e':
							_, dia = var.name.split('_')[1:]
							if int(dia) != n_d-1:
								costo_financiero += var.varValue
					# es sin interés, porque para el costo financiero real
					# se necesita la siguiente linea:
					# costos_financiero_logístico *= interes
				except:
					import pdb;
					pdb.set_trace()
					return -1
				exp_dict[str(rand_seed)][name][str(interes_anual)][str(b)] = [costo_total, costo_financiero]
	return exp_dict

def calcula_delta_std(exp_dict):
	N_seeds = len(exp_dict)
	delta_std = 0.0 # quiero la máxima delta_std
	# para cada exp_setup
	for perfil in ["constant", "V"]:
		for interes_anual in np.linspace(0,10,11):
			for b in range(4):
				# recorrer las seeds
				costos_totales = [exp_dict[str(seed)][perfil][str(interes_anual)][str(b)][0] for seed in range(N_seeds)]
				costos_financi = [exp_dict[str(seed)][perfil][str(interes_anual)][str(b)][1] for seed in range(N_seeds)]
				# calcular las std
				std_total_last = np.std(costos_totales)
				std_finan_last = np.std(costos_financi)
				std_total_prev = np.std(costos_totales[:-1])
				std_finan_prev = np.std(costos_financi[:-1])
				# ver la variación porcentual
				if std_total_last==0 or std_total_prev==0:
					delta_std_total = 0.0
				else:
					delta_std_total = np.abs((std_total_last-std_total_prev)/std_total_last)
				if std_finan_last==0 or std_finan_prev==0:
					delta_std_finan = 0.0
				else:
					delta_std_finan = np.abs((std_finan_last-std_finan_prev)/std_finan_last)
				current_delta_std = max(delta_std_total, delta_std_finan)
				delta_std = max(current_delta_std, delta_std)
	return delta_std


In [7]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [21]:
import numpy as np
import pandas as pd
import sys
#from helpers import *
import json

########################################################################
# Parámetros configurables

n_thr = 8 # cantidad de hilos
N_min = 1 # mínimo de iteraciones por escenario
N_max = 0 # máximo de iteraciones por escenario
solver = 'cuopt' # solver
COLLECTION_MULT = 100.0

# experimento actual
exp_id = "exp_2.json"

########################################################################
# Parámetros fijos

n_s = 4 # número de sucursales
n_p = 8 # número de rutas
# matriz de rutas
rutas = np.array([\
[1.0, 0.0, 0.0, 0.0], \
[0.0, 1.0, 0.0, 0.0], \
[0.0, 0.0, 1.0, 0.0], \
[0.0, 0.0, 0.0, 1.0], \
[0.0, 0.0, 1.0, 1.0], \
[0.0, 1.0, 0.0, 1.0], \
[1.0, 0.0, 0.0, 1.0], \
[1.0, 1.0, 1.0, 1.0] \
])
rutas = pd.DataFrame(rutas)

# costos de rutas
#	- Elegirlos de manera que reflejen la idea de distancia en el grafo elegido.
# Grafo
#	    ----A---B
#	  /   / | /
#	R---D---C
# Distancias
#	{A}			R-A-R				2*(1+np.sqrt(2))
#	{B}			R-A-B-A-R			2*(2+np.sqrt(2))
#	{C}			R-D-C-D-R			4
#	{D}			R-D-R				2
#	{C,D}		R-D-C-D-R			4
#	{B,D}		R-D-C-B-C-D-R		2*(2+np.sqrt(2))
#	{A,D}		R-D-A-R				2*(1+np.sqrt(2))
#	{A,B,C,D}	R-D-C-B-A-R			2*(2+np.sqrt(2))

costos_rutas = np.array([\
2*(1+np.sqrt(2)),\
2*(2+np.sqrt(2)),\
4               ,\
2               ,\
4               ,\
2*(2+np.sqrt(2)),\
2*(1+np.sqrt(2)),\
2*(2+np.sqrt(2))\
])

#	El orden de magnitud para el caso naranja fue:
#		1.5e-03 = (total costo logisticio mensual) / (total recaudacion mensual)
# (total recaudacion mensual) = 1

costos_rutas = COLLECTION_MULT*costos_rutas * 1.5e-3 / (4 * np.average(costos_rutas))
costos_rutas = pd.DataFrame(costos_rutas)

# días hábiles
dias_habiles_profile = [1,1,1,1,1,1,0]*4+[1,1]
dias_habiles = np.tile(dias_habiles_profile,(n_p,1))
dias_habiles = pd.DataFrame(dias_habiles)

# ToDo: parámetro que se usa si no todas las sucursales tienen los mismos totales de recaudación
prop_suc = np.ones(n_s)
prop_suc = pd.DataFrame(prop_suc)

# - perfil de recaudación
#	- dos perfiles: constante y con un pico

#collections_profile_constant = np.ones(30)
collections_profile_constant = np.array(dias_habiles_profile)
collections_profile_constant = collections_profile_constant / np.sum(collections_profile_constant)
collections_constant = np.tile(collections_profile_constant,(n_s,1))*COLLECTION_MULT

# ver varianza_diaria.py
#constant_std = 0.025
#constant_std = (1/30)*.525
constant_std = collections_profile_constant[0]*.525*COLLECTION_MULT

collections_profile_V = np.hstack([np.linspace(1,2,10,endpoint=False),np.linspace(2,1,20,endpoint=False)])
collections_profile_V = collections_profile_V*np.array(dias_habiles_profile)
collections_profile_V /= np.sum(collections_profile_V)
collections_V = np.tile(collections_profile_V,(n_s,1))*COLLECTION_MULT

# ver varianza diaria.py
#V_std = 0.01640
#V_std = (1/30)*.3444
V_std = collections_profile_constant[0]*.3444*COLLECTION_MULT

collections_profiles = [collections_constant,collections_V]
std_profiles = [constant_std, V_std]

data_dir = './data/'

# Dias habiles por ruta
habiles_csv_path = os.path.join(data_dir, "habiles.csv")
dias_habiles.to_csv(habiles_csv_path, header=False, index=False)

# Datos de rutas
rutas_csv_path = os.path.join(data_dir, "rutas.csv")
rutas.to_csv(rutas_csv_path, header=False, index=False)

# Costos por ruta
costo_rutas_csv_path = os.path.join(data_dir, "costo_rutas.csv")
costos_rutas.to_csv(costo_rutas_csv_path, header=False, index=False)

########################################################################
# Ejecución de escenarios

# Llaves:
#	- rand_seed (1)
#		- perfil (2)
#			- interés (11) [0,1,2,3,4,5,6,7,8,9,10]
#				- buzón (4)

# Values:
#	- costo total
#	- costo financiero sin interés

# abrir archivo
with open(exp_id,'r',encoding='utf-8') as f:
	exp_dict = json.load(f)

# mientras N < N_min
while len(exp_dict) < N_min:
	# agregar una corrida
	exp_dict = agregar_resultado(exp_dict, collections_profiles, std_profiles, n_thr, solver)
	# guardar dict
	with open(exp_id,'w',encoding='utf-8') as f:
		json.dump(exp_dict,f,indent=2)

# calcular delta_std
delta_std = calcula_delta_std(exp_dict)

# mientras delta_std > 0.01 y N < N_max
while delta_std > 0.01 and len(exp_dict) < N_max:
	# agregar una corrida
	exp_dict = agregar_resultado(exp_dict, collections_profiles, std_profiles, n_thr, solver)
	# guardar dict
	with open(exp_id,'w',encoding='utf-8') as f:
		json.dump(exp_dict,f,indent=2)
	# calcular delta_std
	delta_std = calcula_delta_std(exp_dict)


Resolviendo caso 0 constant 0.0 0 t=0.10
> /tmp/ipython-input-3115513737.py(190)agregar_resultado()
    188                                         import pdb;
    189                                         pdb.set_trace()
--> 190                                         return -1
    191                                 exp_dict[str(rand_seed)][name][str(interes_anual)][str(b)] = [costo_total, costo_financiero]
    192         return exp_dict

ipdb> list(dir(Problems[0]))
*** Error in argument: '(dir(Problems[0]))'
ipdb> type(Problems)
<class 'list'>
ipdb> type(Problems[0])
<class 'pulp.pulp.LpProblem'>
ipdb> dir(Problems[0])
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclassh

TypeError: object of type 'int' has no len()